# Interactive Plotting of Open Flight Data in Bokeh with Tiles

## TODO
* Try with short trail days
* Widget

In [1]:
import bokeh.plotting as plotting
import datetime as dt
import os
import pandas as pd
from bokeh.io import show
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import ZoomInTool, ZoomOutTool,HoverTool
from bokeh.models.glyphs import Circle
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap
from datashader.utils import lnglat_to_meters as webm

plotting.output_notebook()

Loading BokehJS ...

In [2]:
def ticks_to_datetime(ticks, rtn_type='datetime'):
    if rtn_type == 'datetime':
        return dt.datetime.fromtimestamp(ticks / 1000)
    elif rtn_type == 'time_string':
        return dt.datetime.fromtimestamp(ticks / 1000).strftime('%H:%M:%S.%f')
    elif rtn_type == 'time_string_short':
        return dt.datetime.fromtimestamp(ticks / 1000).strftime('%H:%M')
    elif rtn_type == 'datetime_string':
        return dt.datetime.fromtimestamp(ticks / 1000).strftime('%m-%d-%Y %H:%M:%S.%f')
def datetime_to_ticks(dt_):
    return dt.datetime.timestamp(dt_)*1000
now=dt.datetime.now()
assert ticks_to_datetime(datetime_to_ticks(now), 'datetime') == now

In [3]:
h5_dir = '/users/lukestarnes/Documents/adsb'
# h5_dir = r'C:\adsb'
h5_files = [os.path.join(h5_dir, f) for f in os.listdir(h5_dir)]
h5_file = '/users/lukestarnes/Documents/adsb/2018-06-11.h5'
with pd.HDFStore(h5_file) as store:
    columns_to_keep = ['Lat','Long','Alt','PosTime','To','From','Icao','Id','Mdl',
                   'Spd', 'Trak', 'TTrk', 'Man', 'Op']
    condition = '(Icao == "896194") | (Icao == "3445D4")'
    df = store.select('data', where=condition,
                      columns=columns_to_keep).dropna(subset=('Lat','Long'))

In [4]:
w = webm(df.Long, df.Lat)
df.loc[:,'x'] = w[0]
df.loc[:,'y'] = w[1]

In [5]:
legs = df[['From','To']].drop_duplicates().dropna().set_index('From').to_dict()['To']
for i, (f, t) in enumerate(legs.items()):
    t = [t]
    f = [f]
    criteria = "From == @f & To == @t"
    df.loc[df.eval(criteria), 'Leg'] = i + 1

In [6]:
airport_codes_url='https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
airport_codes = pd.read_csv(airport_codes_url,index_col=0)
airport_codes.columns = ['Name','City','Country','IATA','ICAO','Latitude','Longitude',
                         'Altitude','Timezone','DST','Tz','Type','Source']

In [7]:
tooltips = [
    ("ICAO", "@Icao"),
    ("To-From", "@To-@From"),
    ("Lat, Long", "@Lat, @Long"),
    ("Alt", "@Alt'"),
    ('Gnd Speed', '@Spd knts'),
    ('Aircraft', '@Mdl'),
    ('Airline', '@Op'),
    ('Heading','@Trak deg'),
    ('Time', """@TimeString
    <style>
        .bk-tooltip>div:not(:first-child) {display:none;}
        </style>""")]
#https://stackoverflow.com/a/51249293/6334587

In [8]:
legs = df[['From','To']].drop_duplicates().dropna().set_index('From').to_dict()['To']
for i, (f, t) in enumerate(legs.items()):
    print(f'Leg {i+1}:')
    df_leg = df[(df['From'] == f) & (df['To'] == t)]
    from_airport = airport_codes[airport_codes['ICAO'] == f].Name.values[0]
    to_airport = airport_codes[airport_codes['ICAO'] == t].Name.values[0]
    departure_time = ticks_to_datetime(df_leg.PosTime.min(),'time_string_short')
    arrival_time = ticks_to_datetime(df_leg.PosTime.max(),'time_string_short')
    print(f'Departed {from_airport} at {departure_time}Z and arrived at {to_airport} at {arrival_time}Z')

Leg 1:
Departed Adolfo Suárez Madrid–Barajas Airport at 14:13Z and arrived at Brussels Airport at 16:00Z
Leg 2:
Departed Dubai International Airport at 07:15Z and arrived at London Heathrow Airport at 10:55Z
Leg 3:
Departed London Heathrow Airport at 15:11Z and arrived at Dubai International Airport at 18:12Z


In [9]:
plot_tile = plotting.figure(x_axis_type="mercator", y_axis_type="mercator")
plot_tile.match_aspect = True
mapper = linear_cmap(field_name='Leg', palette=Spectral6 ,low=df.Leg.min() ,high=df.Leg.max())
circle = Circle(x="x", y="y", size=3, line_color=None, fill_color=mapper, fill_alpha=0.9)
plot_tile.add_glyph(ColumnDataSource(df), circle)

icoas = ', '.join(list(df.Icao.value_counts().index))
plot_tile.title.text = f"Flights on ICOAs {icoas}"
plot_tile.add_tools(ZoomInTool(),ZoomOutTool(),HoverTool(tooltips=tooltips))
plot_tile.axis.visible = False

In [13]:
# source: https://leaflet-extras.github.io/leaflet-providers/preview/
Esri_NatGeoWorldMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
Esri_OceanBasemap = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}'
CartoDB_Positron = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}{r}.png'
CartoDB_Voyager = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/rastertiles/voyager/{z}/{x}/{y}{r}.png'
OpenStreetMap_Mapnik = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
OpenTopoMap = 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'
Hydda_Full = 'https://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'
Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_WorldTopoMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}'
Esri_WorldImagery = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

In [15]:
plot_tile.add_tile(WMTSTileSource(url=Esri_WorldStreetMap, snap_to_zoom=True))
show(plot_tile)